# AI Data Analysis
**Carson Case: carsonpcase@gmail.com**

### Instructions in README

In [15]:
# Set up env variables
import os
ASTRA_DB_BUNDLE_PATH =os.getenv("ASTRA_DB_BUNDLE_PATH")
ASTRA_DB_TOKEN = os.getenv("TOKEN")
ASTRA_DB_CLIENT_ID = os.getenv("CLIENT_ID")
ASTRA_DB_CLIENT_SECRET = os.getenv("SECRET")
ASTRA_DB_KEYSPACE = os.getenv("KEYSPACE")
OPENAI_KEY= os.getenv("OPENAI_KEY")

In [16]:
# Import libraries
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

from datasets import load_dataset, Dataset

In [17]:
# Config Astra
clound_config = {
    "secure_connect_bundle":  ASTRA_DB_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider(ASTRA_DB_CLIENT_ID, ASTRA_DB_CLIENT_SECRET)
cluster = Cluster(cloud=clound_config, auth_provider=auth_provider)
astra_session = cluster.connect()

llm = OpenAI(openai_api_key=OPENAI_KEY)
myEmbedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

In [19]:
# Create Cassandra Store and table if it doesn't exist
listingCassandraStore = Cassandra(
    embedding=myEmbedding,
    session=astra_session,
    keyspace=ASTRA_DB_KEYSPACE,
    table_name="listings"
)

In [20]:
# Import listing data
import sqlite3
import pandas as pd

def get_listings():
    """Returns a list of all listings."""
    # Connect to the database.
    db = sqlite3.connect("db/listings.db")
    cursor = db.cursor()

    # Get all listings from the database.
    cursor.execute("SELECT * FROM listings")

    # Create a list of all listings.
    listings = []
    for row in cursor.fetchall():
        listings.append(row)

    # Close the connection to the database.
    db.close()

    # Return the list of listings.
    return listings

# retrieve listings from sqlite
items = get_listings()

# create dataframe and clean it up
df = pd.DataFrame(items, columns=["id", "title","price", "link"])
df["price"] = df["price"].apply(lambda p: int(p.replace('$', '').replace(',', '')))
df = df[["title", "price", "link"]]
df = df[df["price"] != 0]

# sort decending price
df = df.sort_values("price", ascending=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101999 entries, 36036 to 88968
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   101999 non-null  object
 1   price   101999 non-null  int64 
 2   link    101999 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB


In [27]:
docs = []
# Set Size of chunks to publish. Larger chunks make embedding much faster 
chunksize = 100
# Set Size to embed. OpenAI will limit tokens per minute, chunking will not help you here and you may have to do at separate times
sizeToEmbed = int(len(df)/4)

for i in range(0, sizeToEmbed, chunksize):
    textDoc = ""
    for j in range(i, (i+chunksize) % sizeToEmbed):
        row = df.iloc[j]
        textDoc += row["title"] + ": $"+str(row["price"]) +"\n"
    docs.append(Document(page_content=textDoc))

inserted_ids = listingCassandraStore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")
print(f"\n Embedded {sizeToEmbed}/{len(df)} listings")


Inserted 255 documents.

 Embedded 25499/101999 listings


In [29]:
vectorIndex = VectorStoreIndexWrapper(vectorstore=listingCassandraStore)

query = "What is a normal price for a Mercedes e350?"
answer = vectorIndex.query(question=query, llm=llm).strip()

print(answer)

print("Docs by relevance")
for doc, score in listingCassandraStore.similarity_search_with_score(query, k=4):
    print("Score:\t",score,"\n",doc)

The price listed above is for a 2007 Mercedes-Benz E-Class 4dr Sdn 6.3L AMG RWD and is $16995. A normal price for a Mercedes e350 may vary depending on the year, model, and condition of the vehicle.
Docs by relevance
Score:	 0.9315984957609272 
 page_content='2007 Mercedes-Benz E-Class 4dr Sdn 6.3L AMG RWD: $16995'
Score:	 0.9315984957609272 
 page_content='2007 Mercedes-Benz E-Class 4dr Sdn 6.3L AMG RWD: $16995'
Score:	 0.9315828684726786 
 page_content='2007 Mercedes-Benz E-Class 4dr Sdn 6.3L AMG RWD: $16995'
Score:	 0.9315671865588695 
 page_content='2007 Mercedes-Benz E-Class 4dr Sdn 6.3L AMG RWD: $16995'
